In [ ]:
%%script bash
mkdir scrapers/
mkdir scrapers/amazon
cd scrapers/amazon
python -m pip install virtualenv
virtualenv env
. env/bin/activate
python -m pip install scrapy
python -m pip install numpy
python -m pip install pandas
python -m pip install requests
python -m pip install scraperapi-sdk
scrapy startproject amazon_scraper

In [ ]:
%%script bash
python -m pip install pandas

In [3]:
import pandas as pd
data = pd.read_csv('store_credentials .csv', header=0, sep=';', index_col=False)
data

,Email,Password
0,song.of.traitors@gmail.com,13fd46Dnm77


In [4]:
%%writefile settings.py
# -*- coding: utf-8 -*-

# Scrapy settings for amazon_scraper project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'amazon_scraper'

SPIDER_MODULES = ['amazon_scraper.spiders']
NEWSPIDER_MODULE = 'amazon_scraper.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Safari/605.1.15'

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://docs.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
#DOWNLOAD_DELAY = 3
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
#DEFAULT_REQUEST_HEADERS = {
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#   'Accept-Language': 'en',
#}

# Enable or disable spider middlewares
# See https://docs.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'amazon_scraper.middlewares.AmazonScraperSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'amazon_scraper.middlewares.AmazonScraperDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://docs.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
#ITEM_PIPELINES = {
#    'amazon_scraper.pipelines.AmazonScraperPipeline': 300,
#}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'

Writing settings.py


In [5]:
%%script bash
mv settings.py scrapers/amazon/amazon_scraper/amazon_scraper/

In [6]:
%%writefile amazon_spider.py
import scrapy
import numpy as np
import pandas as pd
#from scraper_api import ScraperAPIClient

class AmazonSpider(scrapy.Spider):
    name = 'amazon'
    #client = ScraperAPIClient('yourAPI')
    start_urls = [client.scrapyGet(url=''),]
    url_data = np.array([])
    url_counter = 0

    def parse(self, response):
        links =\
        response.css('div.organic__path a::attr(href)').getall()
        self.url_data = np.append(self.url_data, np.array(links))
        self.url_counter+=len(links)     
        next_page = response.css('a.pager__item_kind_next::attr(href)').get()
        if (self.url_counter<2500) and (next_page is not None):
            yield scrapy.Request(self.client.scrapyGet(url = self.search_engine+next_page), self.parse)
        
    def closed(self, reason):
        print(self.url_counter)
        raw = pd.DataFrame(data=self.url_data)
        raw.to_csv(r'/home/jovyan/raw_url.csv', index = False, header=False)

Writing amazon_spider.py


In [ ]:
%%script bash 
mv amazon_spider.py scrapers/amazon/amazon_scraper/amazon_scraper/spiders

In [ ]:
%%script bash
cd scrapers/amazon
. env/bin/activate
cd amazon_scraper/
scrapy crawl amazon